# Парсер статей ГК РФ

In [223]:
from bs4 import BeautifulSoup
import pathlib, re, pandas as pd

In [224]:
def read_file_text(path: pathlib.Path) -> str:
    """Reads a .doc file saved as HTML and returns plain text"""
    data = path.read_bytes()

    if b'<html' in data.lower():
        soup = BeautifulSoup(data, 'html.parser')
        text = soup.get_text('\n')
    else:
        try:
            text = data.decode('utf-8')
        except UnicodeDecodeError:
            text = data.decode('cp1251', errors='ignore')
    return text.replace('\u00A0', ' ')

In [225]:
# Regular Expression Patterns
re_section = re.compile(r'^\s*(?:\d+\s*\.\s*)?РАЗДЕЛ\s+([IVXLCDM]+)\.?\s*(.*)$',re.I)
re_chapter = re.compile(r'^Глава\s+(\d+)\.?\s*(.*)$', re.I | re.M)
re_article = re.compile(r'^Статья\s+(\d+)\.?\s*(.*)$', re.I | re.M)

In [226]:
# Roman Numeral Converter
def roman_to_int(s: str) -> int:
    vals = dict(I=1, V=5, X=10, L=50, C=100, D=500, M=1000)
    total = 0
    prev = 0
    for ch in reversed(s.upper()):
        v = vals.get(ch, 0)
        total += -v if v < prev else v
        prev = v
        
    return total or None

In [227]:
def parse_articles(path: pathlib.Path):

    def _save_article():
        """Saves the accumulated article in articles_meta"""
        article_end_char = sum(len(l) for l in lines[:i])
        rule_text = text[article_start_char:article_end_char]
        articles_meta.append({
            'file': path.name,
            'rule_number': rule_number,
            'rule_title': rule_title,
            'rule_text': rule_text,
            'section_title': current_section_title,
            'chapter_title': current_chapter_title,
            'start_char': article_start_char,
            'end_char': article_end_char,
            'text_length': len(rule_text)
        })

    def _peek_for_title(lines, start_idx):
        """
        Returns the first "meaningful" line after 'Article N'.
        Ignores empty, '1', '1.', '.', '2.', etc.
        """
        for ln in lines[start_idx:]:
            candidate = ln.strip()
            if candidate == '':
                continue
            if re.match(r'^[\d.]+$', candidate):
                continue
            # trim possible '1.' at the beginning of the line (our bugfix)
            candidate = re.sub(r'^\d+\s*\.?\s*', '', candidate).strip()
            if candidate:
                return candidate
        return ''
    
    def _peek_for_header(lines, start_idx):
        """
        Returns the first "content" line after the section/chapter/article title, ignoring:
        • empty lines; 
        • '1', '1.', 'n1', 'n2', '.'.
        """
        for ln in lines[start_idx:]:
            candidate = ln.strip()
            if not candidate:
                continue
            # n1 / n2 / 1 / 1. / .  => пропускаем
            if re.match(r'^(n?\d+|\\.)\\s*\\.?$', candidate, re.I):
                continue
            # убираем ведущие '1.' и т.п.
            candidate = re.sub(r'^\\d+\\s*\\.?\\s*', '', candidate).strip()
            if candidate:
                return candidate
        return ''


    text = read_file_text(path)
    lines = text.splitlines(keepends=True)

    articles_meta = []
    current_section_title = ''
    current_chapter_title = ''
    rule_number = None
    rule_title = ''
    article_start_char = None

    for i, raw_line in enumerate(lines):
        line = raw_line.strip()

        # --- раздел -------------------------------------
        m_sec = re_section.match(line)
        if m_sec:
            if rule_number is not None:
                _save_article()
            roman = m_sec.group(1).upper() 
            raw_title = m_sec.group(2).strip()
            current_section_title = (
                raw_title if raw_title else f"Раздел {roman}"
            )


        # --- глава --------------------------------------
        m_ch = re_chapter.match(line)
        if m_ch:
            if rule_number is not None:
                _save_article()

            chapter_num = int(m_ch.group(1))
            current_chapter_title = m_ch.group(2).strip()

            # if empty - we search further
            if current_chapter_title == '':
                current_chapter_title = _peek_for_header(lines, i + 1)

            # a backup option if you can't find anything at all
            if current_chapter_title == '':
                current_chapter_title = f'Глава {chapter_num}'


        # --- статья -------------------------------------
        m_art = re_article.match(line)
        if m_art:
            # close the previous one
            if rule_number is not None:
                _save_article()

            rule_number = int(m_art.group(1))
            rule_title = m_art.group(2).strip()

            # bugfix: if the title is empty, we search further
            if rule_title == '':
                rule_title = _peek_for_title(lines, i + 1)

            article_start_char = sum(len(l) for l in lines[:i])

    if rule_number is not None:
        _save_article()

    return articles_meta

In [228]:
data_dir = pathlib.Path('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset')
files = sorted(data_dir.glob('Part*.doc'))  # Search Part1.doc, Part2.doc, Part3.doc, Part4.doc

In [229]:
files

[PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part1.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part2.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part3.doc'),
 PosixPath('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part4.doc')]

In [230]:
all_articles = []
for f in files:
    all_articles.extend(parse_articles(f))

df = pd.DataFrame(all_articles)
print(df.head())

        file  rule_number                                         rule_title  \
0  Part1.doc            1      Основные начала гражданского законодательства   
1  Part1.doc            2  Отношения, регулируемые гражданским законодате...   
2  Part1.doc            3  Гражданское законодательство и иные акты, соде...   
3  Part1.doc            4  Действие гражданского законодательства во времени   
4  Part1.doc            5                                             Обычаи   

                                           rule_text    section_title  \
0  Статья 1. Основные начала гражданского законод...  Общие положения   
1  Статья 2. Отношения, регулируемые гражданским ...  Общие положения   
2  Статья 3. Гражданское законодательство и иные ...  Общие положения   
3  Статья 4. Действие гражданского законодательст...  Общие положения   
4  Статья 5. Обычаи \n(Наименование в редакции Фе...  Общие положения   

                  chapter_title  start_char  end_char  text_length  
0  Гражданс

## Check if data was parsed correctly

In [231]:
print("Shape: ", df.shape)
print("Columns: ", df.columns)

Shape:  (1815, 9)
Columns:  Index(['file', 'rule_number', 'rule_title', 'rule_text', 'section_title',
       'chapter_title', 'start_char', 'end_char', 'text_length'],
      dtype='object')


In [232]:
for col in df.columns:
    print(f"Уникальные значения в столбце '{col}':")
    print(df[col].unique())
    print("-" * 40)


Уникальные значения в столбце 'file':
['Part1.doc' 'Part2.doc' 'Part3.doc' 'Part4.doc']
----------------------------------------
Уникальные значения в столбце 'rule_number':
[   1    2    3 ... 1539 1540 1541]
----------------------------------------
Уникальные значения в столбце 'rule_title':
['Основные начала гражданского законодательства'
 'Отношения, регулируемые гражданским законодательством'
 'Гражданское законодательство и иные акты, содержащие нормы гражданского права'
 ... 'Исключительное право на коммерческое обозначение'
 'Действие исключительного права на коммерческое обозначение'
 'Соотношение права на коммерческое обозначение с правами на фирменное наименование и товарный знак']
----------------------------------------
Уникальные значения в столбце 'rule_text':
['Статья 1. Основные начала гражданского законодательства\n \n1. Гражданское законодательство основывается на признании равенства участников регулируемых им отношений, неприкосновенности собственности, свободы дого

In [233]:
# Checking for empty or NaN lines
print(df.isnull().sum())

for col in df.columns:
    num_empty = (df[col].astype(str).str.strip() == '').sum()
    print(f"Пустых строк в '{col}': {num_empty}")


file             0
rule_number      0
rule_title       0
rule_text        0
section_title    0
chapter_title    0
start_char       0
end_char         0
text_length      0
dtype: int64
Пустых строк в 'file': 0
Пустых строк в 'rule_number': 0
Пустых строк в 'rule_title': 0
Пустых строк в 'rule_text': 0
Пустых строк в 'section_title': 0
Пустых строк в 'chapter_title': 0
Пустых строк в 'start_char': 0
Пустых строк в 'end_char': 0
Пустых строк в 'text_length': 0


In [234]:
empty_titles = df[df['chapter_title'].astype(str).str.strip() == '']
print(empty_titles[['rule_title', 'rule_text', 'section_title']].iloc[:10])

Empty DataFrame
Columns: [rule_title, rule_text, section_title]
Index: []


In [235]:
counts = df[df['chapter_title'].astype(str).str.strip() == ''].groupby('file').size()
print(counts)


Series([], dtype: int64)


In [236]:
text = read_file_text(pathlib.Path('/Users/theother_archee/CursorProjects/SmartClause/parser/dataset/Part4.doc'))
for line in text.splitlines():
    if 'chapter_title' in line.lower():
        print(repr(line))


In [237]:
# Let's calculate the average line length for each column
str_cols = df.select_dtypes(include=['object']).columns
avg_str_lengths = df[str_cols].apply(lambda col: col.astype(str).str.len().mean())

print(avg_str_lengths)

file                9.000000
rule_title         50.751515
rule_text        1203.493113
section_title      23.511295
chapter_title      25.651240
dtype: float64


In [238]:
df[df['file'] == 'Part2.doc'].sort_values('text_length', ascending=False).head()

,file,rule_number,rule_title,rule_text,section_title,chapter_title,start_char,end_char,text_length
1050,Part2.doc,859,Расторжение договора банковского счета,Статья 859. Расторжение договора банковского с...,Отдельные виды обязательств,Банковский счет,343995,348195,4200
1081,Part2.doc,871,Исполнение аккредитива,Статья 871. Исполнение аккредитива\n \n1. Испо...,Отдельные виды обязательств,Расчеты,381203,384322,3119
1076,Part2.doc,867,Общие положения о расчетах по аккредитиву,Статья 867. Общие положения о расчетах по аккр...,Отдельные виды обязательств,Расчеты,374235,377302,3067
1318,Part2.doc,1086,"Определение заработка (дохода), утраченного в ...","Статья 1086. Определение заработка (дохода), у...",Отдельные виды обязательств,Обязательства вследствие причинения вреда,578341,581348,3007
702,Part2.doc,529,Заключение договора поставки товаров для госуд...,Статья 529. Заключение договора поставки товар...,Отдельные виды обязательств,Купля-продажа,74624,77516,2892


## Save the final CSV

In [239]:
csv_path = data_dir / 'dataset_gk_rf.csv'
df.to_csv(csv_path, index=False)
print(f'CSV saved: {csv_path}')

CSV saved: /Users/theother_archee/CursorProjects/SmartClause/parser/dataset/dataset_gk_rf.csv
